In [2]:
!pip install mediapipe

In [8]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

In [9]:
model_path = "pose_landmarker_lite.task"

In [10]:
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a pose landmarker instance with the video mode:
options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO,
    min_pose_detection_confidence=0.5,
    min_pose_presence_confidence=0.5,
    min_tracking_confidence=0.5,
)

In [14]:
with PoseLandmarker.create_from_options(options) as landmarker:
    # read a video file
    cap = cv2.VideoCapture("jenna-ps-video.mp4")
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0:
        fps = 30  # Fallback in case FPS info is missing
    timestamp_ms = 0
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # 6. Convert BGR→RGB and wrap as mp.Image
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image  = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

        # 7. Run detection for this timestamp
        result = landmarker.detect_for_video(mp_image, int(timestamp_ms))

        # 8. Draw the first (or only) pose's landmarks
        annotated = frame.copy()
        if result.pose_landmarks:
            for pose_landmarks in result.pose_landmarks:
                for lm in pose_landmarks:
                    x_px = int(lm.x * annotated.shape[1])
                    y_px = int(lm.y * annotated.shape[0])
                    cv2.circle(annotated, (x_px, y_px), 5, (0,255,0), -1)

        # 9. Display
        cv2.imshow("Pose Landmarker", annotated)
        if cv2.waitKey(int(1000/fps)) & 0xFF == ord('q'):
            break

        timestamp_ms += 1000.0 / fps

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1747326501.807994 1346002 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M2
W0000 00:00:1747326501.937510 1377329 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747326501.958284 1377328 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
